## Unspent Hasher PayOut (UHPO) Mechanism

In this NoteBook, we tried to implement all 3 types of Transactions from UHPO model.

Reference: https://gist.github.com/pool2win/77bb9b98f9f3b8c0f90963343c3c840f

![alt text](https://gist.githubusercontent.com/pool2win/77bb9b98f9f3b8c0f90963343c3c840f/raw/8fa2481728e7c12d553608af23ca6e551c7c90c1/uhpo-success.png "UHPO")



### Tx - Flow
- ALICE , BOB , CHARLIE , DAVID create their respective 1-4 blocks
- `payoutUpdate` is submitted on Time for 1-3 blocks after coinbase maturity
- `payoutUpdate` for coinbase 4 is not submitted on time, hence `payoutsettlement` is submitted to chain.
- After timelock expiry for coinbase 4 bob redeems timelock funds 

In [ ]:
transition :setup do
    # reset regtest node
    node :reset
    
    # creating new key-pairs 
    @alice = key :new 
    @bob = key :new
    @charlie = key :new
    @david = key :new
    
    # FROST keypairs for respective blocks
    @poolkey_1 = key :new
    @poolkey_2 = key :new
    @poolkey_3 = key :new
    @poolkey_4 = key :new
end

In [ ]:
# miners can only spend funds after `x` blocks of coinbase maturity
@alice_timelock = "10 OP_CSV OP_DROP @alice OP_CHECKSIG"

# coinbase outputs are taproot address where script_pub_key = FROST_POOL_PUBKEY + G*( MAST )
# where MAST has a depth of level 1 with only timelock contract
# funds can either be spent via frost-signatures or timelock script path

transition :create_first_block  do
    extend_chain taproot: { internal_key: @poolkey_1, leaves: [ @alice_timelock]}

    @poolkey_101 = key :new
    @poolkey_timelock_101 = key :new 
    @timelock_101 = "10 OP_CSV OP_DROP @poolkey_timelock_101 OP_CHECKSIG"
    @alice_coinbase_1 = get_coinbase_at 1
    

    
    @payout_update_101 = transaction inputs: [
                                    {
                                        tx: @alice_coinbase_1,
                                        vout: 0,
                                        script_sig: { keypath: @poolkey_1 } 
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_101, leaves: [ @timelock_101 ] },                                                    
                                           amount: 49.999.sats
                                         }
                                    ]
    
    # #keypath spend
    @payout_settlement_keypath_101 = transaction inputs: [
                                 { 
                                    tx: @payout_update_101,
                                     vout: 0,
                                     script_sig: { keypath: @poolkey_1 }, 
                                     sighash: :all,
                                }
                            ],
                            outputs: [
                                 { descriptor: 'wpkh(@alice)', amount: 30.sats },
                                 { descriptor: 'wpkh(@charlie)', amount: 10.sats },
                                 { descriptor: 'wpkh(@david)', amount: 9.980.sats } 
                    ]

end

In [ ]:
@bob_timelock = "10 OP_CSV OP_DROP @bob OP_CHECKSIG"

transition :create_second_block do
    extend_chain taproot: { internal_key: @poolkey_2, leaves: [ @bob_timelock]}

    @bob_coinbase_2 = get_coinbase_at 2

    @poolkey_timelock_102 = key :new 
    @timelock_102 = "10 OP_CSV OP_DROP @poolkey_timelock_102 OP_CHECKSIG"

    @poolkey_102 = key :new 
    @payout_update_102 = transaction inputs: [
                                    {
                                        tx: @bob_coinbase_2,
                                        vout: 0,
                                        script_sig: { keypath: @poolkey_2 }
                                    }, { 
                                        tx: @payout_update_101,
                                         vout: 0,
                                         script_sig: { keypath: @poolkey_101 }, 
                                         sighash: :all
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_102, leaves: [ @timelock_102 ] }, 
                                           amount: 99.990.sats
                                         }
                                    ]

     #keypath spend
    @payout_settlement_keypath_102 = transaction inputs: [
                                 { 
                                    tx: @payout_update_102,
                                     vout: 0,
                                     script_sig: { keypath: @poolkey_2 }, 
                                     sighash: :all,
                                }
                            ],
                            outputs: [
                                 { descriptor: 'wpkh(@alice)', amount: 40.sats },
                                 { descriptor: 'wpkh(@bob)', amount: 20.sats },
                                 { descriptor: 'wpkh(@charlie)', amount: 10.sats },
                                 { descriptor: 'wpkh(@david)', amount: 9.980.sats } 
                    ]
end

In [ ]:
@charlie_timelock = "10 OP_CSV OP_DROP @charlie OP_CHECKSIG" 

transition :create_third_block_frost_succeeds do
    extend_chain taproot: { internal_key: @poolkey_3, leaves: [ @charlie_timelock]}
    @charlie_coinbase_3 = get_coinbase_at 3

    @poolkey_timelock_103 = key :new 
    @timelock_103 = "10 OP_CSV OP_DROP @poolkey_timelock_103 OP_CHECKSIG"

    @poolkey_103 = key :new 
    @payout_update_103 = transaction inputs: [
                                    {
                                        tx: @charlie_coinbase_3,
                                        vout: 0,
                                        script_sig: { keypath: @poolkey_3 }
                                    }, { 
                                        tx: @payout_update_102,
                                         vout: 0,
                                         script_sig: { keypath: @poolkey_102 }, 
                                         sighash: :all
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_103, leaves: [ @timelock_103 ] }, 
                                           amount: 149.980.sats
                                         }
                                    ]

    # key-path  spend
    @payout_settlement_keypath_103 = transaction inputs: [
                             { 
                                tx: @payout_update_103,
                                 vout: 0,
                                 script_sig: { keypath: @poolkey_103 }, 
                                 sighash: :all,
                            }
                        ],
                        outputs: [
                             { descriptor: 'wpkh(@alice)', amount: 30.sats },
                             { descriptor: 'wpkh(@alice)', amount: 40.sats },
                             { descriptor: 'wpkh(@alice)', amount: 30.sats },
                             { descriptor: 'wpkh(@alice)', amount: 49.970.sats } 
                            ]
end

In [ ]:
@charlie_timelock = "10 OP_CSV OP_DROP @charlie OP_CHECKSIG" 

transition :create_third_block_frost_fails do
    extend_chain taproot: { internal_key: @poolkey_3, leaves: [ @charlie_timelock]}
    @charlie_coinbase_3 = get_coinbase_at 3

    @poolkey_timelock_103 = key :new 
    @timelock_103 = "10 OP_CSV OP_DROP @poolkey_timelock_103 OP_CHECKSIG"

    @poolkey_103 = key :new 
    
    @payout_update_103 = transaction inputs: [
                                    {
                                        tx: @charlie_coinbase_3,
                                        vout: 0,
                                        script_sig: { keypath: @poolkey_3 }
                                    }, { 
                                        tx: @payout_update_102,
                                         vout: 0,
                                         script_sig: { keypath: @poolkey_102 }, 
                                         sighash: :all
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_103, leaves: [ @timelock_103 ] }, 
                                           amount: 149.980.sats
                                         }
                                    ]

    # case Frost-signature aggregation failed
    # using timelock spend

    @payout_settlement__timelock_103 = transaction inputs: [
                                 { 
                                    tx: @payout_update_103,
                                     vout: 0,
                                     script_sig: { leaf_index: 0, sig: 'sig:@poolkey_timelock_103' }, 
                                     sighash: :all,
                                     csv: 10
                                }
                            ],
                            outputs: [
                                 { descriptor: 'wpkh(@alice)', amount: 30.sats },
                                 { descriptor: 'wpkh(@alice)', amount: 40.sats },
                                 { descriptor: 'wpkh(@alice)', amount: 30.sats },
                                 { descriptor: 'wpkh(@alice)', amount: 49.970.sats } 
                                ]
end

In [ ]:
# case: this transition is for fail-case.
# Braidpool couldn't build/broadcast payout update for block 4
# possible reasons:
    # - threshold miners of braidpool are unreachable or got corrupted
# hence david redeems funds after timelock

transition :create_fourth_block do
    @david_timelock = "10 OP_CSV OP_DROP @david OP_CHECKSIG"
    extend_chain taproot: { internal_key: @poolkey_4, leaves: [ @david_timelock]}
end

In [ ]:

transition :submit_payout_update_for_block_1 do
    broadcast @payout_update_101
    confirm transaction: @payout_update_101

end


In [ ]:
# payout update transaction which merges past payout funds and coinbase block 2 funds

transition :submit_payout_update_for_block_2 do    
    broadcast @payout_update_102
    confirm transaction: @payout_update_102

    assert_not_mempool_accept @payout_settlement_keypath_101
end

In [ ]:
# payout update transaction which merges past payout funds and coinbase block 3 funds

transition :submit_payout_update_for_block_3 do
    broadcast @payout_update_103
    confirm transaction: @payout_update_103

    assert_not_mempool_accept @payout_settlement_keypath_102

end

In [ ]:
# case: payout-settlement (i.e; miners requested for cashout!!)

transition :submit_payout_settlement_keypath do
    broadcast @payout_settlement_keypath_103
    confirm transaction: @payout_settlement_keypath_103
end


In [ ]:
# case: payout-settlement (i.e; payoutUpdate for coinbase of 4th block is not submitted in time `10 blocks` )

transition :submit_payout_settlement_after_timelock do
    extend_chain num_blocks: 10
    broadcast @payout_settlement__timelock_103
    confirm transaction: @payout_settlement__timelock_103
end


In [ ]:
# fail-case since payout update is not submitted in time bob can spend funds from coinbase 4 after 10 more blocks of maturity

transition :david_spends_funds_after_timeout do
    extend_chain num_blocks: 10

    @david_coinbase_4 = get_coinbase_at 4
    @david_failure_close_tx = transaction inputs: [
                                    {
                                        tx: @david_coinbase_4,
                                        vout: 0,
                                        script_sig: { leaf_index: 0, sig: 'sig:@david' },
                                        csv: 10
                                    }
                                ],
                                outputs: [
                                         {
                                           taproot: { internal_key: @poolkey_101, leaves: [] }, 
                                           amount: 49.999.sats
                                         }
                                    ]
    
    broadcast @david_failure_close_tx
    confirm transaction: @david_failure_close_tx
end


In [ ]:
# happy-flow of execution in braidpool
run_transitions :setup,
                :create_first_block,
                :create_second_block,
                :create_third_block_frost_succeeds

extend_chain num_blocks: 97
assert_height 100

run_transitions :submit_payout_update_for_block_1,
                :submit_payout_update_for_block_2,
                :submit_payout_update_for_block_3,
                :submit_payout_settlement_keypath

In [ ]:
# fail-case
# 1: couldnt construct FROST in time, hence funds are redeemed into timelock key in order to spend
# 2: david's coinbase wasnt able to be merged with existing payout-update uhpo txs hence david redeems after timelock

run_transitions :setup,
                :create_first_block,
                :create_second_block,
                :create_third_block_frost_fails,
                :create_fourth_block

extend_chain num_blocks: 96
assert_height 100

run_transitions :submit_payout_update_for_block_1,
                :submit_payout_update_for_block_2,
                :submit_payout_update_for_block_3,
                :submit_payout_settlement_after_timelock,
                :david_spends_funds_after_timeout